In [ ]:
import os
import cv2
import torch
from facenet_pytorch import MTCNN
from PIL import Image
from tqdm import tqdm

In [ ]:
# Device configuration (use GPU if available)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
# Function to process a single video with face tracking
def process_video(args):
    video_path, output_dir = args

    try:
        # Create a VideoCapture object for the input video
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            return

        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        if frame_count == 0:
            return

        video_name = os.path.splitext(os.path.basename(video_path))[0]
        video_output_dir = os.path.join(output_dir, video_name)
        os.makedirs(video_output_dir, exist_ok=True)

        # Initialize MTCNN and tracking variables
        mtcnn = MTCNN(
            image_size=299,
            margin=0,
            min_face_size=20,
            thresholds=[0.6, 0.7, 0.7],
            post_process=False,
            device=device
        )
        tracker = None
        tracking_initialized = False
        frame_idx = -1
        saved_frame_idx = 0
        bbox = None  # Initialize bbox

        # Create a progress bar for each video
        pbar = tqdm(total=frame_count, desc=f"Processing {video_name}", leave=False)

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame_idx += 1
            pbar.update(1)

            # Update tracker on every frame if initialized
            if tracking_initialized:
                success, bbox = tracker.update(frame)
                if not success:
                    tracking_initialized = False  # Tracker failed, re-initialize
                    bbox = None  # Reset bbox

            # If tracking is not initialized, attempt face detection
            if not tracking_initialized:
                img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                img_pil = Image.fromarray(img_rgb)
                boxes, _ = mtcnn.detect(img_pil)
                if boxes is not None:
                    # Use the first detected face to initialize the tracker
                    x1, y1, x2, y2 = [int(b) for b in boxes[0]]
                    tracker = cv2.TrackerCSRT_create()
                    tracker.init(frame, (x1, y1, x2 - x1, y2 - y1))
                    bbox = (x1, y1, x2 - x1, y2 - y1)  # Assign initial bbox
                    tracking_initialized = True
                else:
                    bbox = None  # Reset bbox
                    continue  # Skip to the next frame if no face is detected

            # Save face images for every frame where tracking is successful
            if tracking_initialized and bbox is not None:
                x, y, w, h = [int(v) for v in bbox]

                # Expand the bounding box by a factor of 1.2 to add margin
                margin_factor = 1.2  # Using margin factor of 1.2
                center_x = x + w / 2
                center_y = y + h / 2
                new_w = w * margin_factor
                new_h = h * margin_factor
                new_x = center_x - new_w / 2
                new_y = center_y - new_h / 2

                # Ensure bounding box is within frame dimensions
                height, width = frame.shape[:2]
                new_x = max(0, int(new_x))
                new_y = max(0, int(new_y))
                new_w = min(int(new_w), width - new_x)
                new_h = min(int(new_h), height - new_y)

                # Crop and resize the face region with margin
                face = frame[new_y:new_y+new_h, new_x:new_x+new_w]
                face = cv2.resize(face, (299, 299))

                # Save the face image
                output_filename = f"frame_{saved_frame_idx:04d}.jpg"
                output_filepath = os.path.join(video_output_dir, output_filename)
                cv2.imwrite(output_filepath, face)
                saved_frame_idx += 1

        pbar.close()
        cap.release()
        # print(f"Extracted {saved_frame_idx} frames from {video_name}")

    except Exception as e:
        print(f"Error processing video {video_path} at frame {frame_idx}: {e}")


In [ ]:
# Function to preprocess all videos in the input directories
def preprocess_videos(input_dirs, output_dir):
    video_files = []
    for input_dir in input_dirs:
        for root, _, files in os.walk(input_dir):
            for file in files:
                if file.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
                    video_path = os.path.join(root, file)
                    video_files.append(video_path)

    os.makedirs(output_dir, exist_ok=True)

    # Prepare arguments for processing
    tasks = []
    output_dirs = []
    for video_file in video_files:
        video_name = os.path.splitext(os.path.basename(video_file))[0]
        video_output_dir = os.path.join(output_dir, video_name)
        tasks.append((video_file, output_dir))
        output_dirs.append(video_output_dir)

    # Process videos sequentially with overall progress bar
    num_tasks = len(tasks)
    # print(f"Found {num_tasks} videos to process")

    for idx, args in enumerate(tqdm(tasks, total=num_tasks, desc='Overall Progress')):
        video_file, _ = args
        video_name = os.path.splitext(os.path.basename(video_file))[0]
        video_output_dir = output_dirs[idx]

        if idx < num_tasks - 1:
            # For all but the last video, skip if output folder exists
            if os.path.exists(video_output_dir):
                print(f"Skipping {video_name} - already processed")
                continue  # Skip processing
        else:
            # For the last video, delete output folder if it exists and reprocess
            if os.path.exists(video_output_dir):
                print(f"Reprocessing last video: {video_name}")
                # Delete the existing output folder
                try:
                    for root, dirs, files in os.walk(video_output_dir, topdown=False):
                        for name in files:
                            os.remove(os.path.join(root, name))
                        for name in dirs:
                            os.rmdir(os.path.join(root, name))
                    os.rmdir(video_output_dir)
                except Exception as e:
                    print(f"Error cleaning up directory: {e}")

        process_video(args)

In [ ]:
if __name__ == "__main__":
    # Specify your input directories and output directory
    input_dirs = [
        r'/content/drive/MyDrive/Dataset DDM/FF++/manipulated_sequences/FaceShifter/raw/videos',   # Replace with your first input directory path
        # Replace with your second input directory path
    ]

    output_dir = r'/content/drive/MyDrive/Dataset DDM/Processed_FF+/Fake_NEW_faceShiter'  # Replace with your single output directory path

    preprocess_videos(input_dirs, output_dir)